# Training Notebook

In [1]:
#Imports

import os

import rasterio
from rasterio.plot import reshape_as_image
import rasterio.mask
from rasterio.features import rasterize

import pandas as pd
import geopandas as gpd
from shapely.geometry import mapping, Point, Polygon
from shapely.ops import cascaded_union

import numpy as np
import cv2
import matplotlib.pyplot as plt

import torch

import os
import numpy as np
import torch
from torch.utils.data import Dataset, random_split, DataLoader
import torch.nn as nn

import models

In [2]:
class SegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.image_list = [f for f in os.listdir(image_dir) if f.endswith('.npy')]

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        image_path = os.path.join(self.image_dir, self.image_list[idx])
        mask_path = os.path.join(self.mask_dir, self.image_list[idx])

        image = np.load(image_path)
        mask = np.load(mask_path)

        # Convert numpy arrays to PyTorch tensors
        image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1)
        mask = torch.tensor(mask, dtype=torch.float32) # Assuming masks are integer labels

        return image, mask

image_dir = 'semantic_filtered_data/images/'
mask_dir = 'semantic_filtered_data/masks/'

print("normal stuff done")
dataset = SegmentationDataset(image_dir=image_dir, mask_dir=mask_dir)

# Splitting the dataset into train, val, and test sets
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

normal stuff done
Train dataset size: 153
Validation dataset size: 32
Test dataset size: 34


In [3]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)
test_loader = DataLoader(test_dataset, batch_size=8)

In [5]:
device = torch.device("cpu")

## LOADING EUNET 
#model = models.U_Net(img_ch=3,output_ch=1)
model = models.EdgeU1_Net(img_ch=3,output_ch=1)
        
# Defining  loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
#criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters())
#optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
num_epochs = 30

# Set the device to use for training
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


######################## Training model on train dataset #####################################

# Train the model
model.to(device)

#Tracking Losses
losst_list = []
lossv_list = []

for epoch in range(num_epochs):
    train_loss = 0.0
    
    #Train the model
    model.train()
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        #print(inputs.shape)
        labels = labels.to(device).unsqueeze(1) 
        #print('labels are')
        #print(labels.shape)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and update
        loss.backward()
        optimizer.step()

        # Print statistics
        train_loss += loss.item()
    avg_train_loss = train_loss / len(train_loader)
    losst_list.append(avg_train_loss)
            
    #Validate the training process
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs = inputs.to(device)
            targets = targets.to(device).unsqueeze(1) 
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()
        avg_val_loss = val_loss / len(val_loader)
    lossv_list.append(avg_val_loss)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")
    
        
        #print(f"Epoch number{i}")
        #print(f"[Accuracy]{evalmetrics.get_accuracy(outputs,labels,threshold=0.5)}")
        #print(f"[Sensitivity]{evalmetrics.get_sensitivity(outputs,labels,threshold=0.5)}")
        #print(f"[Specificity]{evalmetrics.get_specificity(outputs,labels,threshold=0.5)}")
        #print(f"[Precision]{evalmetrics.get_precision(outputs,labels,threshold=0.5)}")
        #print(f"[F1]{evalmetrics.get_F1(outputs,labels,threshold=0.5)}")
        #print(f"[JS]{evalmetrics.get_JS(outputs,labels,threshold=0.5)}") 
        #print(f"[DC]{evalmetrics.get_DC(outputs,labels,threshold=0.5)}")

: 